In [7]:
# TensorFlow and tf.keras
import tensorflow as tf
import tensorflowjs as tfjs

# Helper libraries
import numpy as np
# import matplotlib.pyplot as plt

import os, cv2, glob, json, base64

# from PIL import Image
from blend_modes import normal


# Lets make sure working directory is project root
if os.getcwd().rsplit(os.path.sep,1)[1]=="model_training":
    os.chdir( os.getcwd().rsplit(os.path.sep,1)[0] )

# Load up our lookup_table used for labels
with open(os.path.join("public","lookup_table.json")) as json_file:
    lookup_table  = json.load(json_file)

# Define functions used throughout the document
def read_png(path):
    image = cv2.imread(path,-1)
    return cv2.cvtColor(image,cv2.COLOR_BGR2BGRA) if len(image.shape) > 2 and image.shape[2] == 3 else image

def read_jpeg(path):
    BGR = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    BGRA = cv2.cvtColor(BGR,cv2.COLOR_BGR2BGRA)
    BGRA[:,:,3] = 255
    return BGRA

def layer( image_a, image_b ):
    return normal(image_a.astype(float), image_b.astype(float), 1.).astype(np.uint8)

def save_model(model, model_name):
    model.save                             (os.path.join("model_training","models",model_name,"model.h5"))
    model.save                             (os.path.join("model_training","models",model_name,"model"))
    model.save_weights                     (os.path.join("model_training","models",model_name,"model_weights_checkpoint","model_weights"))
    tfjs.converters.save_keras_model(model, os.path.join("public","models",model_name))
    

# Generate champion and skin training images

In [11]:
def generate_champion_and_skin_training_images( champion_image_path ):
    
    champion_id = champion_image_path.rsplit(os.path.sep,1)[1].split('.')[0]
    champion_data = list(filter(lambda row: row[0] == champion_id, lookup_table["champions"]+lookup_table["skins"]))[0]
    price = champion_data[2]
    is_legacy = champion_data[3] == 1
    
    # 1.A loading and preparing assets
    champion = cv2.copyMakeBorder(read_jpeg(champion_image_path), 0, 5, 5, 0, cv2.BORDER_CONSTANT,value=(0,0,0,255))
    shard_border = cv2.resize(read_png(os.path.join("model_training","assets","border_images","shard.png")), (385,385))
    permanent_border = cv2.resize(read_png(os.path.join("model_training","assets","border_images","permanent.png")), (385,385))
    token_6_border = read_png(os.path.join("model_training","assets","border_images","champion_token_6.png"))[5:74, :]
    token_6_border = cv2.resize(token_6_border, (385,385))
    token_7_border = read_png(os.path.join("model_training","assets","border_images","champion_token_7.png"))[5:74, :]
    token_7_border = cv2.resize(token_7_border, (385,385))
    quantity_background = cv2.resize(read_png(os.path.join("model_training","assets","border_images","quantity_background.png")), (385,385))

    if is_legacy: legacy_icon = read_png(os.path.join("model_training","assets","tag_icons","legacy.png"))
        
    if price == "special": rarity_icon = read_png(os.path.join("model_training","assets","rarity_icons","rarity_mythic.png"))
    if price == 3250: rarity_icon = read_png(os.path.join("model_training","assets","rarity_icons","rarity_ultimate.png"))
    if price == 1820: rarity_icon = read_png(os.path.join("model_training","assets","rarity_icons","rarity_legendary.png"))
    if price == 1350: rarity_icon = read_png(os.path.join("model_training","assets","rarity_icons","rarity_epic.png"))

    # 1.B Resize rarity and legacy icons to 395x395 pixel resolution overlay images
    if 'legacy_icon' in locals():
        legacy_icon = cv2.resize(legacy_icon, (110,110))
        legacy_icon = cv2.copyMakeBorder(legacy_icon, 291, 0, 0, 297, cv2.BORDER_CONSTANT,value=(0,0,0,0))
        legacy_icon = legacy_icon[0:395,legacy_icon.shape[1]-395:legacy_icon.shape[1]]

    if 'rarity_icon' in locals():
        rarity_icon = cv2.resize(rarity_icon, (385,int(385/rarity_icon.shape[1]*rarity_icon.shape[0])))
        rarity_icon = cv2.copyMakeBorder(rarity_icon, 285, 5, 5, 5, cv2.BORDER_CONSTANT,value=(0,0,0,0))
        rarity_icon = rarity_icon[0:395,0:395]

    # 2. Set up borders this asset uses in the loot tab
    if champion_id.endswith("000"):
        borders = (shard_border, permanent_border, token_6_border, token_7_border)
    else:
        borders = (shard_border, permanent_border)
    
    # 3. Create variations of assets from layers
    for border_index in range(len(borders)):
        for shadow in (False, True):
            image = layer(champion, quantity_background) if shadow else champion
            image = layer(image, borders[border_index])
            image = cv2.copyMakeBorder(image, 5, 5, 5, 5, cv2.BORDER_CONSTANT,value=(14,14,14,255))            

            y_range_increment = 2
            if not champion_id.endswith("000"):
                y_range_increment = 1
                if 'legacy_icon' in locals(): image = layer(image, legacy_icon)  
                if 'rarity_icon' in locals(): image = layer(image, rarity_icon)  
                    
            image = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)

          
            # Create different crops to accomodate the imperfection of client screenshot percision
            for x in range(0, 13, 2):
                for y in range(1, 13, y_range_increment):
                    crop = image[0+y:image.shape[1]-10+y, 0+x:image.shape[1]-10+x]              
                    resized_image = cv2.resize(crop, (28,28), interpolation = cv2.INTER_AREA)
                    
                    # plt.figure()
                    # plt.imshow(resized_image)
                    # plt.colorbar()
                    # plt.show()

                    
                    model_folder = "champions" if champion_id.endswith("000") else "skins"                        
                    destination = os.path.join("model_training","training_data",model_folder,champion_id,f'{("shard","permanent","token6","token7")[border_index]}_{"yes" if shadow else "no"}-shadow_x{x}_y{y}.png')

                    if not os.path.exists(destination.rsplit(os.path.sep,1)[0]):
                        os.makedirs(destination.rsplit(os.path.sep,1)[0])

                    cv2.imwrite(destination, resized_image)

In [12]:
# Get champion and skins assets
champions_and_skins = glob.glob(os.path.join("model_training","assets","champions_and_skins","*.jpg"))

# Generate training data if the lookup_table contains the ID
# If ID is missing in lookup table run "generate_lookup_table.py"
# If ID is still missing, this champion is very fresh as wikipedia doesn't know of him :)
missing_ids = []

for image in champions_and_skins:
    champion_id = image.rsplit(os.path.sep,1)[1].split('.')[0]
    champion_data = list(filter(lambda row: row[0] == champion_id, lookup_table["champions"]+lookup_table["skins"]))
    if len(champion_data) > 0:
        print("Generating training data for: "+champion_data[0][1])
        generate_champion_and_skin_training_images(image)
    else:
        missing_ids.append(champion_id)
        print("Warning: Champion with ID: ["+champion_id+"] couldn't be found.")

if len(missing_ids) > 0:
    print("#################################################")
    print("Missing IDs (read comments in code for more info)")
    for id in missing_ids:
        print(id)


Generating training data for: Annie
Generating training data for: Kayle
Generating training data for: Silver Kayle
Generating training data for: Viridian Kayle
Generating training data for: Transcended Kayle
Generating training data for: Battleborn Kayle
Generating training data for: Judgment Kayle
Generating training data for: Aether Wing Kayle
Generating training data for: Riot Kayle
Generating training data for: Iron Inquisitor Kayle
Generating training data for: Pentakill Kayle
Generating training data for: Goth Annie
Generating training data for: PsyOps Kayle
Generating training data for: Red Riding Annie
Generating training data for: Dragonslayer Kayle
Generating training data for: Annie in Wonderland
Generating training data for: Prom Queen Annie
Generating training data for: Frostfire Annie
Generating training data for: Reverse Annie
Generating training data for: FrankenTibbers Annie
Generating training data for: Panda Annie
Generating training data for: Sweetheart Annie
Genera

Generating training data for: Beast Hunter Draven
Generating training data for: Draven Draven
Generating training data for: Santa Draven
Generating training data for: Mecha Kingdoms Draven
Generating training data for: Ruined Draven
Generating training data for: Alistar
Generating training data for: Hecarim
Generating training data for: Blood Knight Hecarim
Generating training data for: Reaper Hecarim
Generating training data for: Headless Hecarim
Generating training data for: Arcade Hecarim
Generating training data for: Elderwood Hecarim
Generating training data for: Worldbreaker Hecarim
Generating training data for: Lancer Zero Hecarim
Generating training data for: High Noon Hecarim
Generating training data for: Black Alistar
Generating training data for: Cosmic Charger Hecarim
Generating training data for: Golden Alistar
Generating training data for: Matador Alistar
Generating training data for: Longhorn Alistar
Generating training data for: Unchained Alistar
Generating training dat

Generating training data for: Order of the Banana Soraka
Generating training data for: Program Soraka
Generating training data for: Star Guardian Soraka
Generating training data for: Pajama Guardian Soraka
Generating training data for: Winter Wonder Soraka
Generating training data for: Dawnbringer Soraka
Generating training data for: Nightbringer Soraka
Generating training data for: Prestige Star Guardian Soraka
Generating training data for: Vel'Koz
Generating training data for: Battlecast Vel'Koz
Generating training data for: Arclight Vel'Koz
Generating training data for: Definitely Not Vel'Koz
Generating training data for: Infernal Vel'Koz
Generating training data for: Blackfrost Vel'Koz
Generating training data for: Taliyah
Generating training data for: Freljord Taliyah
Generating training data for: SSG Taliyah
Generating training data for: Pool Party Taliyah
Generating training data for: Camille
Generating training data for: Program Camille
Generating training data for: Coven Camil

Generating training data for: Championship Zed
Generating training data for: Death Sworn Zed
Generating training data for: Galaxy Slayer Zed
Generating training data for: PsyOps Zed
Generating training data for: Jax
Generating training data for: Kled
Generating training data for: Sir Kled
Generating training data for: Count Kledula
Generating training data for: Marauder Kled
Generating training data for: The Mighty Jax
Generating training data for: Vandal Jax
Generating training data for: Angler Jax
Generating training data for: PAX Jax
Generating training data for: Jaximus
Generating training data for: Temple Jax
Generating training data for: Nemesis Jax
Generating training data for: SKT T1 Jax
Generating training data for: Warden Jax
Generating training data for: God Staff Jax
Generating training data for: Mecha Kingdoms Jax
Generating training data for: Conqueror Jax
Generating training data for: Prestige Conqueror Jax
Generating training data for: Ekko
Generating training data for:

Generating training data for: Heartseeker Yuumi
Generating training data for: Yuubee Yuumi
Generating training data for: Royal Shaco
Generating training data for: Nutcracko Shaco
Generating training data for: Workshop Shaco
Generating training data for: Asylum Shaco
Generating training data for: Masked Shaco
Generating training data for: Wild Card Shaco
Generating training data for: Dark Star Shaco
Generating training data for: Arcanist Shaco
Generating training data for: Dr. Mundo
Generating training data for: Samira
Generating training data for: PsyOps Samira
Generating training data for: Toxic Dr. Mundo
Generating training data for: Space Groove Samira
Generating training data for: Mr. Mundoverse Dr. Mundo
Generating training data for: Corporate Mundo Dr. Mundo
Generating training data for: Mundo Mundo Dr. Mundo
Generating training data for: Executioner Mundo Dr. Mundo
Generating training data for: Rageborn Mundo Dr. Mundo
Generating training data for: TPA Mundo Dr. Mundo
Generating

Generating training data for: Elderwood Rakan
Generating training data for: Xayah
Generating training data for: Cosmic Dusk Xayah
Generating training data for: Sweetheart Xayah
Generating training data for: SSG Xayah
Generating training data for: Star Guardian Xayah
Generating training data for: Elderwood Xayah
Generating training data for: Xin Zhao
Generating training data for: Swain
Generating training data for: Northern Front Swain
Generating training data for: Bilgewater Swain
Generating training data for: Tyrant Swain
Generating training data for: Dragon Master Swain
Generating training data for: Commando Xin Zhao
Generating training data for: Hextech Swain
Generating training data for: Crystal Rose Swain
Generating training data for: Imperial Xin Zhao
Generating training data for: Viscero Xin Zhao
Generating training data for: Winged Hussar Xin Zhao
Generating training data for: Warring Kingdoms Xin Zhao
Generating training data for: Secret Agent Xin Zhao
Generating training data

Generating training data for: Brand
Generating training data for: Apocalyptic Brand
Generating training data for: Vandal Brand
Generating training data for: Cryocore Brand
Generating training data for: Zombie Brand
Generating training data for: Spirit Fire Brand
Generating training data for: Battle Boss Brand
Generating training data for: Arclight Brand
Generating training data for: Eternal Dragon Brand
Generating training data for: Lee Sin
Generating training data for: Traditional Lee Sin
Generating training data for: Acolyte Lee Sin
Generating training data for: Dragon Fist Lee Sin
Generating training data for: Muay Thai Lee Sin
Generating training data for: Pool Party Lee Sin
Generating training data for: SKT T1 Lee Sin
Generating training data for: Knockout Lee Sin
Generating training data for: God Fist Lee Sin
Generating training data for: Playmaker Lee Sin
Generating training data for: Nightbringer Lee Sin
Generating training data for: Prestige Nightbringer Lee Sin
Generating tra

Generating training data for: PROJECT Mordekaiser
Generating training data for: Yorick
Generating training data for: Undertaker Yorick
Generating training data for: Pentakill Yorick
Generating training data for: Arclight Yorick
Generating training data for: Meowrick Yorick
Generating training data for: Resistance Yorick
Generating training data for: Akali
Generating training data for: Stinger Akali
Generating training data for: Infernal Akali
Generating training data for: All-star Akali
Generating training data for: Nurse Akali
Generating training data for: Blood Moon Akali
Generating training data for: Silverfang Akali
Generating training data for: Headhunter Akali
Generating training data for: Sashimi Akali
Generating training data for: KDA Akali
Generating training data for: Prestige KDA Akali
Generating training data for: PROJECT Akali
Generating training data for: True Damage Akali
Generating training data for: KDA ALL OUT Akali
Generating training data for: Kennen
Generating trai

# Generate ward skin training images

In [48]:
def generate_ward_skin_training_images( ward_image_path ):

    # 1.A loading and preparing assets
    ward = read_png(ward_image_path)
    ward = cv2.copyMakeBorder(ward[0:ward.shape[1],:], 0, 7, 7, 0, cv2.BORDER_CONSTANT,value=(0,0,0,0))
    ward_background = cv2.resize(read_png(os.path.join("model_training","assets","border_images","wardskin_background.png")), (467,467))
    shard_border = cv2.resize(read_png(os.path.join("model_training","assets","border_images","shard.png")), (467,467))
    permanent_border = cv2.resize(read_png(os.path.join("model_training","assets","border_images","permanent.png")), (467,467))
    quantity_background = cv2.resize(read_png(os.path.join("model_training","assets","border_images","quantity_background.png")), (467,467))

    ward_id = ward_image_path.rsplit("_",1)[1].split(".")[0]

    borders = (shard_border, permanent_border)
        
    # 2. Create variations of assets from layers
    for border_index in range(len(borders)):
        for shadow in (False, True):
            image = layer(ward_background, quantity_background) if shadow else ward_background
            image = layer(image, ward)
            image = layer(image, borders[border_index])
            image = cv2.copyMakeBorder(image, 5, 5, 5, 5, cv2.BORDER_CONSTANT,value=(14,14,14,255))
                    
            image = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)
        
            # Create different crops to accomodate the imperfection of client screenshot percision
            for x in range(0, 13, 2):
                for y in range(1, 13, 1):
                    crop = image[0+y:image.shape[1]-10+y, 0+x:image.shape[1]-10+x]              
                    resized_image = cv2.resize(crop, (28,28), interpolation = cv2.INTER_AREA)

                    destination = os.path.join("model_training","training_data","wards",ward_id,f'{("shard","permanent")[border_index]}_{"yes" if shadow else "no"}-shadow_x{x}_y{y}.png')
                    if not os.path.exists(destination.rsplit(os.path.sep,1)[0]):
                        os.makedirs(destination.rsplit(os.path.sep,1)[0])

                    cv2.imwrite(destination, resized_image)

In [49]:
# Get ward skin assets while ignoring their shadow png image
ward_skins = glob.glob(os.path.join("model_training","assets","ward_skins","*.png"))
ward_skins = list(filter(lambda ward: "shadow" not in ward, ward_skins))

# Generate training data if the lookup_table contains the ID
# If ID is missing in lookup table run "generate_lookup_table.py"
# If ID is still missing, this champion is very fresh as wikipedia doesn't know of him :)
missing_ids = []

for image in ward_skins:
    ward_id = image.rsplit("_",1)[1].split(".")[0]
    ward_data = list(filter(lambda row: row[0] == int(ward_id), lookup_table["wards"]))
    if len(ward_data) > 0:
        print("Generating training data for: "+ward_data[0][1])
        generate_ward_skin_training_images(image)
    else:
        missing_ids.append(ward_id)
        print("Warning: Ward skin with ID: ["+ward_id+"] couldn't be found. Ignored.")

if len(missing_ids) > 0:
    print("#################################################")
    print("Missing IDs (read comments in code for more info)")
    for id in missing_ids:
        print(id)


Generating training data for: Default Ward
Generating training data for: Bat-o-lantern Ward
Generating training data for: Starcall Ward
Generating training data for: Death Sworn Ward
Generating training data for: Essence Collector Ward
Generating training data for: Puppy Warwick Ward
Generating training data for: Puppy Nasus Ward
Generating training data for: Exo-Ward
Generating training data for: Pizza Chicken Ward
Generating training data for: 2018 Conqueror Ward
Generating training data for: Pulsefire 2018 Ward
Generating training data for: The Ripper Ward
Generating training data for: The Lion Ward
Generating training data for: Ward of Draven
Generating training data for: The Wolf Ward
Generating training data for: World Rune Ward
Generating training data for: Surfin' Pengu Ward
Generating training data for: Wipeout Pengu Ward
Generating training data for: Hang Loose Pengu Ward
Generating training data for: Space Lizard Ward
Generating training data for: Devil Vulture Ward
Generati

Generating training data for: Game On Ward
Generating training data for: Star Guardian Ward
Generating training data for: 2016 Championship Ward
Generating training data for: Mechs vs. Minions Ward
Generating training data for: 2016 Conquering Ward
Generating training data for: 2016 Triumphant Ward
Generating training data for: 2016 Victorious Ward
Generating training data for: Elementalist Ward
Generating training data for: Gingerbread Ward
Generating training data for: Vamporo Ward
Generating training data for: Santa Penguin Ward
Generating training data for: Crest of the Rooster Ward
Generating training data for: 2016 All-Star Ward
Generating training data for: Blood Demon Ward
Generating training data for: Love Dove Ward
Generating training data for: Eye of the Dragon Ward
Generating training data for: Pulsefire Ward
Generating training data for: Dawnbringer Ward
Generating training data for: Nightbringer Ward
Generating training data for: Lantern of the Serpent Ward
Generating tra

# Champion recognition model

In [27]:
champion_train_image_paths = glob.glob( os.path.join("model_training","training_data","champions","**","*.png") )

champion_train_labels = []
champion_train_images = []

for i in range(len(champion_train_image_paths)):
    if i%1000 == 0:
        print("Parsing image data: " + str(i//1000) + "K/" + str(len(champion_train_image_paths)//1000) + "K")

    image = champion_train_image_paths[i]
    asset_id = image.rsplit(os.path.sep, 2)[1]
    
    for row_index in range(len(lookup_table["champions"])):
        if lookup_table["champions"][row_index][0] == asset_id:
            label = row_index

    image = read_png(image)
    image = image / 255

    champion_train_labels.append(label)
    champion_train_images.append(image)
    
champion_train_images = np.array(champion_train_images)
champion_train_labels = np.array(champion_train_labels)

champion_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(len(lookup_table["champions"]))
])

champion_model.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

champion_model.fit( champion_train_images, champion_train_labels, epochs=100)

champion_model.summary()
save_model(champion_model, "champions")

Epoch 1/100
1628/1628 [==============================] - 1s 766us/step - loss: 2.3721e-08 - accuracy: 1.0000
Epoch 2/100
1628/1628 [==============================] - 1s 769us/step - loss: 2.3347e-08 - accuracy: 1.0000
Epoch 3/100
1628/1628 [==============================] - 1s 778us/step - loss: 2.3233e-08 - accuracy: 1.0000
Epoch 4/100
1628/1628 [==============================] - 1s 750us/step - loss: 2.2938e-08 - accuracy: 1.0000
Epoch 5/100
1628/1628 [==============================] - 1s 774us/step - loss: 2.2773e-08 - accuracy: 1.0000
Epoch 6/100
1628/1628 [==============================] - 1s 739us/step - loss: 2.2501e-08 - accuracy: 1.0000
Epoch 7/100
1628/1628 [==============================] - 1s 744us/step - loss: 2.2338e-08 - accuracy: 1.0000
Epoch 8/100
1628/1628 [==============================] - 1s 747us/step - loss: 2.2089e-08 - accuracy: 1.0000
Epoch 9/100
1628/1628 [==============================] - 1s 743us/step - loss: 2.1869e-08 - accuracy: 1.0000
Epoch 10/100
1628/1

1628/1628 [==============================] - 1s 745us/step - loss: 1.4393e-08 - accuracy: 1.0000
Epoch 76/100
1628/1628 [==============================] - 1s 746us/step - loss: 1.4313e-08 - accuracy: 1.0000
Epoch 77/100
1628/1628 [==============================] - 1s 753us/step - loss: 1.4288e-08 - accuracy: 1.0000
Epoch 78/100
1628/1628 [==============================] - 1s 739us/step - loss: 1.4221e-08 - accuracy: 1.0000
Epoch 79/100
1628/1628 [==============================] - 1s 746us/step - loss: 1.4153e-08 - accuracy: 1.0000
Epoch 80/100
1628/1628 [==============================] - 1s 757us/step - loss: 1.4107e-08 - accuracy: 1.0000
Epoch 81/100
1628/1628 [==============================] - 1s 762us/step - loss: 1.4091e-08 - accuracy: 1.0000
Epoch 82/100
1628/1628 [==============================] - 1s 763us/step - loss: 1.3999e-08 - accuracy: 1.0000
Epoch 83/100
1628/1628 [==============================] - 1s 749us/step - loss: 1.3958e-08 - accuracy: 1.0000
Epoch 84/100
1628/1628 

# Skin recognition model

In [29]:
skin_train_image_paths = glob.glob( os.path.join("model_training","training_data","skins","**","*.png") )

skin_train_labels = []
skin_train_images = []

for i in range(len(skin_train_image_paths)):
    if i%1000 == 0:
        print("Parsing image data: " + str(i//1000) + "K/" + str(len(skin_train_image_paths)//1000) + "K")

    image = skin_train_image_paths[i]
    asset_id = image.rsplit(os.path.sep, 2)[1]
    
    for row_index in range(len(lookup_table["skins"])):
        if lookup_table["skins"][row_index][0] == asset_id:
            label = row_index

    image = read_png(image)
    image = image / 255

    skin_train_labels.append(label)
    skin_train_images.append(image)
    
skin_train_images = np.array(skin_train_images)
skin_train_labels = np.array(skin_train_labels)

skin_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(len(lookup_table["skins"]))
])

skin_model.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

skin_model.fit( skin_train_images, skin_train_labels, epochs=100)

champion_model.summary()
save_model(skin_model, "skins")

Epoch 1/100
12831/12831 [==============================] - 49s 4ms/step - loss: 0.3951 - accuracy: 0.9662
Epoch 2/100
12831/12831 [==============================] - 48s 4ms/step - loss: 7.4012e-04 - accuracy: 0.9998
Epoch 3/100
12831/12831 [==============================] - 50s 4ms/step - loss: 3.0235e-04 - accuracy: 0.9999
Epoch 4/100
12831/12831 [==============================] - 51s 4ms/step - loss: 1.3490e-04 - accuracy: 1.0000
Epoch 5/100
12831/12831 [==============================] - 51s 4ms/step - loss: 9.1511e-05 - accuracy: 1.0000
Epoch 6/100
12831/12831 [==============================] - 50s 4ms/step - loss: 4.8008e-05 - accuracy: 1.0000
Epoch 7/100
12831/12831 [==============================] - 50s 4ms/step - loss: 4.8000e-05 - accuracy: 1.0000
Epoch 8/100
12831/12831 [==============================] - 50s 4ms/step - loss: 2.8861e-05 - accuracy: 1.0000
Epoch 9/100
12831/12831 [==============================] - 51s 4ms/step - loss: 1.8614e-05 - accuracy: 1.0000
Epoch 10/100
1

12831/12831 [==============================] - 63s 5ms/step - loss: 2.3024e-08 - accuracy: 1.0000
Epoch 75/100
12831/12831 [==============================] - 57s 4ms/step - loss: 2.2840e-08 - accuracy: 1.0000
Epoch 76/100
12831/12831 [==============================] - 54s 4ms/step - loss: 2.2616e-08 - accuracy: 1.0000
Epoch 77/100
12831/12831 [==============================] - 53s 4ms/step - loss: 2.2458e-08 - accuracy: 1.0000
Epoch 78/100
12831/12831 [==============================] - 53s 4ms/step - loss: 2.2269e-08 - accuracy: 1.0000
Epoch 79/100
12831/12831 [==============================] - 53s 4ms/step - loss: 2.2095e-08 - accuracy: 1.0000
Epoch 80/100
12831/12831 [==============================] - 55s 4ms/step - loss: 2.1894e-08 - accuracy: 1.0000
Epoch 81/100
12831/12831 [==============================] - 57s 4ms/step - loss: 2.1740e-08 - accuracy: 1.0000
Epoch 82/100
12831/12831 [==============================] - 56s 4ms/step - loss: 2.1578e-08 - accuracy: 1.0000
Epoch 83/100
1

# Ward sking recognition model

In [55]:
ward_train_image_paths = glob.glob( os.path.join("model_training","training_data","wards","**","*.png") )

ward_train_labels = []
ward_train_images = []

for i in range(len(ward_train_image_paths)):
    if i%1000 == 0:
        print("Parsing image data: " + str(i//1000) + "K/" + str(len(ward_train_image_paths)//1000) + "K")

    image = ward_train_image_paths[i]
    asset_id = image.rsplit(os.path.sep, 2)[1]

    image = read_png(image)
    image = image / 255

    ward_train_labels.append(int(asset_id))
    ward_train_images.append(image)
    
ward_train_images = np.array(ward_train_images)
ward_train_labels = np.array(ward_train_labels)

ward_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(len(lookup_table["wards"]))
])

ward_model.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

ward_model.fit( ward_train_images, ward_train_labels, epochs=100)

champion_model.summary()
save_model(ward_model, "wards")

Epoch 1/100
2216/2216 [==============================] - 2s 900us/step - loss: 0.0011 - accuracy: 1.0000
Epoch 2/100
2216/2216 [==============================] - 2s 888us/step - loss: 0.0010 - accuracy: 1.0000
Epoch 3/100
2216/2216 [==============================] - 2s 889us/step - loss: 9.6825e-04 - accuracy: 1.0000
Epoch 4/100
2216/2216 [==============================] - 2s 969us/step - loss: 9.4661e-04 - accuracy: 1.0000
Epoch 5/100
2216/2216 [==============================] - 2s 905us/step - loss: 9.2907e-04 - accuracy: 1.0000
Epoch 6/100
2216/2216 [==============================] - 3s 1ms/step - loss: 8.6134e-04 - accuracy: 1.0000
Epoch 7/100
2216/2216 [==============================] - 2s 1ms/step - loss: 8.4725e-04 - accuracy: 1.0000
Epoch 8/100
2216/2216 [==============================] - 2s 1ms/step - loss: 8.1943e-04 - accuracy: 1.0000
Epoch 9/100
2216/2216 [==============================] - 2s 1ms/step - loss: 7.9935e-04 - accuracy: 1.0000
Epoch 10/100
2216/2216 [===========

2216/2216 [==============================] - 2s 868us/step - loss: 1.1012e-04 - accuracy: 1.0000
Epoch 76/100
2216/2216 [==============================] - 2s 893us/step - loss: 1.0530e-04 - accuracy: 1.0000
Epoch 77/100
2216/2216 [==============================] - 2s 873us/step - loss: 1.0055e-04 - accuracy: 1.0000
Epoch 78/100
2216/2216 [==============================] - 2s 863us/step - loss: 1.0157e-04 - accuracy: 1.0000
Epoch 79/100
2216/2216 [==============================] - 2s 866us/step - loss: 9.4772e-05 - accuracy: 1.0000
Epoch 80/100
2216/2216 [==============================] - 2s 865us/step - loss: 9.5756e-05 - accuracy: 1.0000
Epoch 81/100
2216/2216 [==============================] - 2s 863us/step - loss: 9.2498e-05 - accuracy: 1.0000
Epoch 82/100
2216/2216 [==============================] - 2s 841us/step - loss: 8.8018e-05 - accuracy: 1.0000
Epoch 83/100
2216/2216 [==============================] - 2s 830us/step - loss: 8.4634e-05 - accuracy: 1.0000
Epoch 84/100
2216/2216 

# Loot icon number/count recognition model

In [ ]:
number_train_labels = []
number_train_images = []

numbers_image = "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEYAAACTAQMAAAAJE/1bAAAAAXNSR0IB2cksfwAAAAlwSFlzAAALEwAACxMBAJqcGAAAAAZQTFRFAAAA////pdmf3QAAAhFJREFUeJyNzz9r20AYx/GjGKNBlDNdBA1BQ4aMJwLtmYRGAbVT+h5cktGGZHOoWrtOiDUYTDaHhPY1ZAt0qHHFNYMSz4YMNinemlZ4afDSe05/fFLkNNvDDZ/7fRHGW8Nm/ZVGkLWLTUd5YjnIKmDk4JzJL4xQvYAUcS3VjxRLQSZu6PUjjT9gwzcPVKzrCJObUfvgI90BZSNUDNzgitVCFkHmp0Jumb8Rc6krFIs0dEcoxLjrNlVMdUT/o+gZSg8UN1SeagllUyj5lEKs/YK6DJf5Iq3QUCkFymGGouVeC8Xdf6bacJVecsVNKMW7bvuRSvu5rGhoW1KmoFQjpQaKjzdaKcUl5KeknCksUC5ipT9fWcG9pOKBst71QSmTHnlIIUJpCaUlFDZfYUllEirlCQVlExSNK/m3oXICiisUR1K0lNI/rK1VQdny1EjxYqX/NVKGM+VYKHvTYagYE58rtqwwUBbyFVCMMSjXaWWgMRooi9SOt1ChvPOUjC1ZRcdhUXJLVlFlVpS1JS6CLcWg6C8qT4VyeYrYKlcWhULjIlpaH/kdxn+jPTKYFX2mc5Sr1Y6sfAOFPazsScqNt/BmpvyKlQ+RcioURgbTWPlC7T/3lT4o53MVG5Qdrowpqr5PKE1J+X0NyvdQKUbKWFZ208oaKO17yo+0cktLtVApgtIJlFGg2LFSeYxyFSu3kvIP9ENXILgWa2YAAAAASUVORK5CYII="
encoded_data = numbers_image.split(',')[1]
nparr = np.frombuffer(base64.b64decode(encoded_data), np.uint8)
img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)[:,:,0] / 255

for x in range(0, img.shape[1], 7):
    for y in range(0, img.shape[0], 7):
        crop = img[y:y+7, x:x+7]

        number_train_labels.append(x//7)
        number_train_images.append(crop)

model_number = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(7, 7)),
    tf.keras.layers.Dense(10)
])

model_number.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

number_train_labels = np.array(number_train_labels)
number_train_images = np.array(number_train_images)

model_number.fit( number_train_images, number_train_labels, epochs=5000)

model_number.summary()
save_model(model_number, "model_numbers")

# Shard/Permanent model

In [ ]:
shard_permanent_image_paths = glob.glob( os.path.join("model_training","training_data","**","*.png") )

shard_permanent_train_labels = []
shard_permanent_train_images = []

for i in range(len(shard_permanent_image_paths)):
    if i%1000 == 0:
        print("Progress: " + str(i//1000) + "/" + str(len(shard_permanent_image_paths)//1000))

    image = shard_permanent_image_paths[i]
    label = 0 if "shard" in image else 1

    image = read_png(image) / 255

    shard_permanent_train_labels.append(label)
    shard_permanent_train_images.append(image)

shard_permanent_train_images = np.array(shard_permanent_train_images)
shard_permanent_train_labels = np.array(shard_permanent_train_labels)

model_shard_permanent = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(2)
])

model_shard_permanent.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
    
model_shard_permanent.fit( shard_permanent_train_images, shard_permanent_train_labels, epochs=10)

model_shard_permanent.summary()
save_model(model_shard_permanent, "model_shard_permanent")